# LlamaExtract Usage

In [ ]:
%pip install llama-extract

In [ ]:
# llama-extract is async-first, running the sync code in a notebook requires the use of nest_asyncio
import nest_asyncio

nest_asyncio.apply()

import os

os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."

In [ ]:
from llama_extract import LlamaExtract

extractor = LlamaExtract()

extraction_schema = extractor.infer_schema("Test Schema", [...])

In [ ]:
print(extraction_schema.data_schema)

{'type': 'object', 'properties': {'Contact': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'email': {'type': 'string'}, 'telephone': {'type': 'string'}}}, 'PaymentNotice': {'type': 'object', 'properties': {'investor': {'type': 'object', 'properties': {'name': {'type': 'string'}}}, 'dealCusip': {'type': 'string'}, 'loanShare': {'type': 'string'}, 'reference': {'type': 'string'}, 'bankDetails': {'type': 'object', 'properties': {'aba': {'type': 'string'}, 'bankName': {'type': 'string'}, 'attention': {'type': 'string'}, 'accountNumber': {'type': 'string'}, 'bankToBankInfo': {'type': 'string'}}}, 'paymentType': {'type': 'string'}, 'effectiveDate': {'type': 'string'}, 'facilityCusip': {'type': 'string'}, 'remittedAmount': {'type': 'string'}, 'interestPayment': {'type': 'string'}, 'paymentDescription': {'type': 'string'}}}, 'LoanRepricingNotice': {'type': 'object', 'properties': {'dealIsin': {'type': 'string'}, 'allInRate': {'type': 'string'}, 'dealCusip': {'type': 'string'}, 

In [ ]:
extractions = extractor.extract(extraction_schema.id, [...])

Extracting files: 100%|██████████| 2/2 [01:02<00:00, 31.19s/it]


In [ ]:
print(extractions[0].data)

{'Contact': {'name': 'HASHNEET KAUR', 'email': 'HASHNEET.KAUR@CHASE.COM', 'telephone': '+1-302-634-1253'}, 'PaymentNotice': {'investor': {'name': 'FARALLON CAPITAL (AM) INVESTORS, L.P.'}, 'dealCusip': 'CORELOGIC INC. $4BN 6/4/21 MUCY*', 'loanShare': 'USD 17,773.30', 'reference': 'CORELOGIC INC. $4BN 6/4/21 MUCY*, CME Term SOFR Reference Rates - Effective Repricing', 'bankDetails': {'aba': '021000089', 'bankName': 'CITIBANK NA', 'attention': 'Goldman Sachs & Co New York', 'accountNumber': '30649484', 'bankToBankInfo': 'FFC 002458339 Farallon Capital AM Investors LP'}, 'paymentType': 'Interest Payment', 'effectiveDate': '31-Oct-2023', 'facilityCusip': None, 'remittedAmount': 'USD 132.39', 'interestPayment': 'USD 132.39', 'paymentDescription': 'Your share of the CME Term SOFR Reference Rates - Effective interest payment'}, 'LoanRepricingNotice': {'dealIsin': None, 'allInRate': '8.938600%', 'dealCusip': None, 'rateBasis': None, 'yourShare': 'USD 17,773.30', 'loanAmount': None, 'description